<a href="https://colab.research.google.com/github/Pathway2008/CarPrice/blob/main/train2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 배터리용량을 추정한 다음 다시 모델을 실행했지만 rmse가 낮아지지 않음

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.font_manager as fm
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

from sklearn.base import RegressorMixin, BaseEstimator
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import shap

In [3]:
!apt-get -qq install fonts-nanum


fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설정

Selecting previously unselected package fonts-nanum.
(Reading database ... 123634 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [4]:
train = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/train.csv')
test = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/sample_submission.csv')

In [5]:
ordinal_features = ['제조사', '차량상태', '구동방식','사고이력',  '모델']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [6]:
model_mapping = dict(enumerate(le.classes_))
model_mapping

{0: 'EV6',
 1: 'ID4',
 2: 'ION5',
 3: 'ION6',
 4: 'IONIQ',
 5: 'KNE',
 6: 'M3',
 7: 'MS',
 8: 'MX',
 9: 'MY',
 10: 'Niro',
 11: 'Q4eT',
 12: 'RSeTGT',
 13: 'Soul',
 14: 'Tay',
 15: 'TayCT',
 16: 'TayGTS',
 17: 'eT',
 18: 'i3',
 19: 'i5',
 20: 'iX'}

In [8]:
# iX, EV6 제외
new_train = train[~train['모델'].isin([0, 20])]
rest_train = train[train['모델'].isin([0,20])]

In [10]:
train_not = new_train[~(new_train['배터리용량'].isna())].reset_index(drop=True)
train_na = new_train[(new_train['배터리용량'].isna())].reset_index(drop=True)

In [12]:
# All_data
X_train_all = train_not.drop(columns=['ID', '가격(백만원)','배터리용량'])
y_train_all = train_not['배터리용량']
model_column_1 = train_not['모델']

In [14]:
class SklearnCompatibleRegressor(RegressorMixin, BaseEstimator):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# 모델 정의
xgb_model = SklearnCompatibleRegressor(XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=41))
cat_model = SklearnCompatibleRegressor(CatBoostRegressor(n_estimators=500, learning_rate=0.05, random_state=41, verbose=0))
lgb_model = SklearnCompatibleRegressor(LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=41))

# Voting Regressors 정의
voting_models = {
    'Cat+LGB': VotingRegressor([('cat', cat_model), ('lgb', lgb_model)])
}

In [15]:
n_splits = 5
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=41)

for fold, (train_idx, valid_idx) in enumerate(stratified_kfold.split(X_train_all, model_column_1)):
    print(f"Fold {fold + 1}")

    # 훈련 및 검증 데이터 분리
    X_train_fold = X_train_all.iloc[train_idx]
    X_valid_fold = X_train_all.iloc[valid_idx]
    y_train_fold = y_train_all.iloc[train_idx]
    y_valid_fold = y_train_all.iloc[valid_idx]

    # 개별 모델 학습
    xgb_model.fit(X_train_fold, y_train_fold)
    lgb_model.fit(X_train_fold, y_train_fold)
    cat_model.fit(X_train_fold, y_train_fold)

    # Voting 모델 학습
    for name, voting_model in voting_models.items():
        voting_model.fit(X_train_fold, y_train_fold)

Fold 1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000761 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 304
[LightGBM] [Info] Number of data points in the train set: 3712, number of used features: 8
[LightGBM] [Info] Start training from score 68.647175
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000449 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 304
[LightGBM] [Info] Number of data points in the train set: 3712, number of used features: 8
[LightGBM] [Info] Start training from score 68.647175
Fold 2
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000514 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info

In [20]:
y_pred_xgb = xgb_model.predict(train_na.drop(['ID','가격(백만원)','배터리용량'],axis=1))
y_pred_voting = voting_model.predict(train_na.drop(['ID','가격(백만원)','배터리용량'],axis=1))

In [21]:
train_na

,ID,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
0,TRAIN_0003,0,12,1,NaN,0,21683,3,0,0,99.16
1,TRAIN_0006,5,7,1,NaN,0,19395,3,0,0,83.42
2,TRAIN_0008,5,9,0,NaN,0,2226,8,0,0,74.06
3,TRAIN_0009,0,11,0,NaN,0,3683,7,0,0,59.66
4,TRAIN_0011,6,1,0,NaN,0,1131,8,0,0,40.50
...,...,...,...,...,...,...,...,...,...,...,...
2115,TRAIN_7489,2,2,0,NaN,0,8871,9,0,0,35.83
2116,TRAIN_7490,0,11,0,NaN,0,5794,7,0,0,59.95
2117,TRAIN_7491,3,13,0,NaN,1,5966,10,0,0,16.75
2118,TRAIN_7492,2,2,0,NaN,0,3773,10,0,0,35.95


In [24]:
# 모델 인코딩 정보
model_encoding = {
    0: 'EV6', 1: 'ID4', 2: 'ION5', 3: 'ION6', 4: 'IONIQ', 5: 'KNE', 6: 'M3', 7: 'MS',
    8: 'MX', 9: 'MY', 10: 'Niro', 11: 'Q4eT', 12: 'RSeTGT', 13: 'Soul', 14: 'Tay',
    15: 'TayCT', 16: 'TayGTS', 17: 'eT', 18: 'i3', 19: 'i5', 20: 'iX'
}

# 조건별 제한 값 정의
max_battery_limits = {
    'Niro': 90, 'RSeTGT': 90, 'i5': 90, 'ION6': 90, 'MY': 90, 'Q4eT': 90, 'ID4': 90,
    'TayCT': 90, 'Soul': 90, 'KNE': 90, 'M3': 90, 'i3': 90, 'ION5': 90, 'Tay': 90,
    'TayGTS': 93.4, 'eT': 95, 'MS': 99.8, 'MX': 99.8
}

In [25]:
# '모델' 컬럼을 모델명으로 디코딩
train_na['모델명'] = train_na['모델'].map(model_encoding)

# 조건에 따라 '배터리용량' 컬럼 값 업데이트
train_na['배터리용량'] = [
    min(
        y_pred_xgb[i] if train_na['모델'].iloc[i] == 4 else y_pred_voting[i],  # 조건별 예측값 선택
        max_battery_limits.get(train_na['모델명'].iloc[i], float('inf'))       # 최대값 제한 적용
    )
    for i in range(len(train_na))
]

In [30]:
train_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2120 entries, 0 to 2119
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        2120 non-null   object 
 1   제조사       2120 non-null   int64  
 2   모델        2120 non-null   int64  
 3   차량상태      2120 non-null   int64  
 4   배터리용량     2120 non-null   float64
 5   구동방식      2120 non-null   int64  
 6   주행거리(km)  2120 non-null   int64  
 7   보증기간(년)   2120 non-null   int64  
 8   사고이력      2120 non-null   int64  
 9   연식(년)     2120 non-null   int64  
 10  가격(백만원)   2120 non-null   float64
 11  모델명       2120 non-null   object 
dtypes: float64(2), int64(8), object(2)
memory usage: 198.9+ KB


In [28]:
rest_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 736 entries, 21 to 7480
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        736 non-null    object 
 1   제조사       736 non-null    int64  
 2   모델        736 non-null    int64  
 3   차량상태      736 non-null    int64  
 4   배터리용량     145 non-null    float64
 5   구동방식      736 non-null    int64  
 6   주행거리(km)  736 non-null    int64  
 7   보증기간(년)   736 non-null    int64  
 8   사고이력      736 non-null    int64  
 9   연식(년)     736 non-null    int64  
 10  가격(백만원)   736 non-null    float64
dtypes: float64(2), int64(8), object(1)
memory usage: 69.0+ KB


In [29]:
train_not.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4641 entries, 0 to 4640
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        4641 non-null   object 
 1   제조사       4641 non-null   int64  
 2   모델        4641 non-null   int64  
 3   차량상태      4641 non-null   int64  
 4   배터리용량     4641 non-null   float64
 5   구동방식      4641 non-null   int64  
 6   주행거리(km)  4641 non-null   int64  
 7   보증기간(년)   4641 non-null   int64  
 8   사고이력      4641 non-null   int64  
 9   연식(년)     4641 non-null   int64  
 10  가격(백만원)   4641 non-null   float64
dtypes: float64(2), int64(8), object(1)
memory usage: 399.0+ KB


In [31]:
after_train = pd.concat([train_not,train_na.drop('모델명',axis=1),rest_train],axis=0).reset_index(drop=True)

In [32]:
after_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7497 non-null   object 
 1   제조사       7497 non-null   int64  
 2   모델        7497 non-null   int64  
 3   차량상태      7497 non-null   int64  
 4   배터리용량     6906 non-null   float64
 5   구동방식      7497 non-null   int64  
 6   주행거리(km)  7497 non-null   int64  
 7   보증기간(년)   7497 non-null   int64  
 8   사고이력      7497 non-null   int64  
 9   연식(년)     7497 non-null   int64  
 10  가격(백만원)   7497 non-null   float64
dtypes: float64(2), int64(8), object(1)
memory usage: 644.4+ KB


In [34]:
# All_data
X_train_all = after_train.drop(columns=['ID', '가격(백만원)'])
y_train_all = after_train['가격(백만원)']
model_column_1 = after_train['모델']

In [33]:
class SklearnCompatibleRegressor(RegressorMixin, BaseEstimator):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# 모델 정의
xgb_model = SklearnCompatibleRegressor(XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=41))
cat_model = SklearnCompatibleRegressor(CatBoostRegressor(n_estimators=500, learning_rate=0.05, random_state=41, verbose=0))
lgb_model = SklearnCompatibleRegressor(LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=41))

# Voting Regressors 정의
voting_models = {
    'XGB+Cat': VotingRegressor([('xgb', xgb_model), ('cat', cat_model)]),
    'XGB+LGB': VotingRegressor([('xgb', xgb_model), ('lgb', lgb_model)]),
    'Cat+LGB': VotingRegressor([('cat', cat_model), ('lgb', lgb_model)]),
    'XGB+Cat+LGB': VotingRegressor([('xgb', xgb_model), ('cat', cat_model), ('lgb', lgb_model)])
}

In [35]:
n_splits = 5
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=41)

rmse_results = {
    'XGB': [],
    'CatBoost': [],
    'LGB': [],
    'XGB+Cat': [],
    'XGB+LGB': [],
    'Cat+LGB': [],
    'XGB+Cat+LGB': []
}

# 결과 저장용 리스트
fold_stats_list = []

# Stratified K-Fold 실행
for fold, (train_idx, valid_idx) in enumerate(stratified_kfold.split(X_train_all, model_column_1)):
    print(f"Fold {fold + 1}")

    # 훈련 및 검증 데이터 분리
    X_train_fold = X_train_all.iloc[train_idx]
    X_valid_fold = X_train_all.iloc[valid_idx]
    y_train_fold = y_train_all.iloc[train_idx]
    y_valid_fold = y_train_all.iloc[valid_idx]

    # 개별 모델 학습
    xgb_model.fit(X_train_fold, y_train_fold)
    lgb_model.fit(X_train_fold, y_train_fold)
    cat_model.fit(X_train_fold, y_train_fold)

    # Voting 모델 학습
    for name, voting_model in voting_models.items():
        voting_model.fit(X_train_fold, y_train_fold)

    # 예측 수행
    y_pred_xgb = xgb_model.predict(X_valid_fold)
    y_pred_lgb = lgb_model.predict(X_valid_fold)
    y_pred_cat = cat_model.predict(X_valid_fold)

    # 결과 저장 DataFrame 생성
    result_df = pd.DataFrame(X_valid_fold)
    result_df['가격(백만원)'] = y_valid_fold.values
    result_df['XGB_Pred'] = y_pred_xgb
    result_df['CatBoost_Pred'] = y_pred_cat
    result_df['LGB_Pred'] = y_pred_lgb

    # 각 Voting 모델에 대한 예측
    for name, voting_model in voting_models.items():
        result_df[f'{name}_Pred'] = voting_model.predict(X_valid_fold)

    # RMSE 계산
    result_df['xgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['XGB_Pred'])**2)
    result_df['cat-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)
    result_df['lgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['LGB_Pred'])**2)
    result_df['xgb_cat-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['XGB+Cat_Pred'])**2)
    result_df['xgb_lgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['XGB+LGB_Pred'])**2)
    result_df['cat_lgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['Cat+LGB_Pred'])**2)
    result_df['xgb_cat_lgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['XGB+Cat+LGB_Pred'])**2)

    # 모델별 RMSE 계산
    rmse_results['XGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['XGB_Pred'])))
    rmse_results['CatBoost'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['CatBoost_Pred'])))
    rmse_results['LGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['LGB_Pred'])))
    rmse_results['XGB+Cat'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['XGB+Cat_Pred'])))
    rmse_results['XGB+LGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['XGB+LGB_Pred'])))
    rmse_results['Cat+LGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['Cat+LGB_Pred'])))
    rmse_results['XGB+Cat+LGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['XGB+Cat+LGB_Pred'])))

    # 모델별 통계 계산
    fold_stats = result_df.groupby('모델')[['xgb-real', 'cat-real', 'lgb-real',
                                            'xgb_cat-real', 'xgb_lgb-real',
                                            'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])
    fold_stats_list.append(fold_stats)

Fold 1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000468 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 555
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.323265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000811 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 555
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.323265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000851 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 555
[LightGBM] [Info] Number of data points in the train s

In [36]:
score_41 = pd.DataFrame(rmse_results)
score_41.loc['Mean'] = score_41.mean()
score_41

,XGB,CatBoost,LGB,XGB+Cat,XGB+LGB,Cat+LGB,XGB+Cat+LGB
0,1.429811,1.434756,1.372167,1.392570,1.383732,1.372857,1.373118
1,1.522189,1.430864,1.507401,1.440206,1.490072,1.436776,1.445247
2,1.405611,1.366034,1.350762,1.346451,1.359679,1.327811,1.334692
3,1.267854,1.335531,1.389027,1.266836,1.285502,1.314993,1.274794
4,1.364396,1.380306,1.427418,1.335155,1.363852,1.360975,1.340630
Mean,1.397972,1.389498,1.409355,1.356244,1.376568,1.362683,1.353696


In [37]:
model_stats_mean = pd.concat(fold_stats_list).groupby('모델').mean()
# 'mean' 열만 필터링
mean_columns = [col for col in model_stats_mean.columns if col[1] == 'mean']

# 스타일링 함수 정의
def highlight_minimum(s):
    is_min = s == s.min()  # 최소값 여부 확인
    return ['background-color: lightgreen' if v else '' for v in is_min]

In [38]:
print("\n전체 평균 모델 성능 통계(random41):")
display(model_stats_mean.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
))


전체 평균 모델 성능 통계(random41):
